# Interactive Visualization of Text Summarization Results

Notebook này cung cấp giao diện tương tác để visualize các kết quả huấn luyện và đánh giá từ các phương pháp fine-tuning khác nhau (LoRA, QLoRA, Adapter, và Prompt-tuning).

## Nội dung:
1. Import và thiết lập
2. Đọc dữ liệu đánh giá
3. Visualize ROUGE scores
4. So sánh thời gian huấn luyện
5. So sánh sử dụng VRAM
6. So sánh số lượng tham số
7. Widget tương tác

In [ ]:
# Import các thư viện cần thiết
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact, interactive, fixed, interact_manual, widgets

# Thêm đường dẫn project vào sys.path
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
if BASE_DIR not in sys.path:
    sys.path.insert(0, BASE_DIR)

# Import từ project
from src.config import FIGURES_DIR, TABLES_DIR, ROUGE_METRICS, REPORT_TITLE

# Thiết lập style cho matplotlib
plt.style.use('seaborn')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

# Cấu hình seaborn
sns.set_style("whitegrid")
sns.set_context("notebook")

## Đọc và Chuẩn Bị Dữ Liệu

Chúng ta sẽ đọc dữ liệu đánh giá từ file CSV trong thư mục `report/tables`. File này chứa các metrics như ROUGE scores, thời gian huấn luyện, và thông tin về tài nguyên sử dụng cho mỗi phương pháp fine-tuning.

In [ ]:
# Đọc file evaluation results
results_path = os.path.join(TABLES_DIR, 'evaluation_results.csv')
df = pd.read_csv(results_path)

# Hiển thị thông tin về DataFrame
print("Thông tin về dữ liệu:")
print("-" * 50)
print(df.info())
print("\nMẫu dữ liệu:")
print("-" * 50)
display(df.head())

## Visualize ROUGE Scores

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) là metric chính để đánh giá chất lượng tóm tắt. Chúng ta sẽ tạo:
1. Bar plot so sánh ROUGE-1, ROUGE-2, và ROUGE-L scores
2. Spider/Radar plot để thấy được điểm mạnh/yếu của mỗi phương pháp
3. Interactive plot với Plotly để có thể zoom và hover

In [ ]:
# Tạo bar plot với seaborn
plt.figure(figsize=(12, 6))
df_melted = pd.melt(df, id_vars=['Method'], value_vars=['ROUGE-1', 'ROUGE-2', 'ROUGE-L'],
                    var_name='Metric', value_name='Score')

sns.barplot(data=df_melted, x='Method', y='Score', hue='Metric')
plt.title('ROUGE Scores Comparison Across Methods')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Tạo interactive plot với plotly
fig = go.Figure()
for metric in ['ROUGE-1', 'ROUGE-2', 'ROUGE-L']:
    fig.add_trace(go.Bar(
        name=metric,
        x=df['Method'],
        y=df[metric],
        text=df[metric].round(3),
        textposition='auto',
    ))

fig.update_layout(
    title='Interactive ROUGE Scores Comparison',
    xaxis_title='Method',
    yaxis_title='Score',
    barmode='group',
    width=900,
    height=500
)
fig.show()

## Training Time và Resource Usage

So sánh hiệu quả của các phương pháp dựa trên:
1. Thời gian huấn luyện
2. Sử dụng VRAM
3. Số lượng tham số có thể huấn luyện

In [ ]:
# Tạo subplot với 3 metrics
fig = make_subplots(rows=1, cols=3, 
                    subplot_titles=('Training Time (minutes)', 'VRAM Usage (GB)', 'Trainable Parameters'))

# Training Time
fig.add_trace(
    go.Bar(name='Training Time', x=df['Method'], y=df['Training Time (min)'],
           text=df['Training Time (min)'].round(1), textposition='auto'),
    row=1, col=1
)

# VRAM Usage
fig.add_trace(
    go.Bar(name='VRAM', x=df['Method'], y=df['VRAM (GB)'],
           text=df['VRAM (GB)'].round(1), textposition='auto'),
    row=1, col=2
)

# Trainable Parameters
fig.add_trace(
    go.Bar(name='Parameters', x=df['Method'], y=df['Trainable Params (num)'] / 1e6,
           text=(df['Trainable Params (num)'] / 1e6).round(1).astype(str) + 'M', 
           textposition='auto'),
    row=1, col=3
)

fig.update_layout(height=400, width=1200, showlegend=False, title_text="Resource Usage Comparison")
fig.update_xaxes(tickangle=45)
fig.show()

## Interactive Comparison

Widget tương tác để so sánh các phương pháp dựa trên các metrics khác nhau. Bạn có thể:
1. Chọn metrics muốn so sánh
2. Lọc các phương pháp cần so sánh
3. Thay đổi loại biểu đồ

In [ ]:
def plot_comparison(metrics, methods, plot_type='bar'):
    """
    Interactive plot function
    """
    if not isinstance(metrics, list):
        metrics = [metrics]
    if not isinstance(methods, list):
        methods = [methods]
        
    df_filtered = df[df['Method'].isin(methods)]
    
    if plot_type == 'bar':
        fig = go.Figure()
        for metric in metrics:
            fig.add_trace(go.Bar(
                name=metric,
                x=df_filtered['Method'],
                y=df_filtered[metric],
                text=df_filtered[metric].round(3),
                textposition='auto',
            ))
        fig.update_layout(barmode='group')
    else:  # radar plot
        fig = go.Figure()
        for method in methods:
            method_data = df[df['Method'] == method]
            fig.add_trace(go.Scatterpolar(
                r=method_data[metrics].values[0],
                theta=metrics,
                fill='toself',
                name=method
            ))
            
    fig.update_layout(
        title=f'Comparison of {", ".join(metrics)}',
        width=800,
        height=500
    )
    fig.show()

# Create widgets
metric_options = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU', 
                 'Training Time (min)', 'VRAM (GB)', 'Trainable Params (num)']
method_options = df['Method'].unique().tolist()

metrics_widget = widgets.SelectMultiple(
    options=metric_options,
    value=['ROUGE-1', 'ROUGE-2', 'ROUGE-L'],
    description='Metrics:',
    disabled=False
)

methods_widget = widgets.SelectMultiple(
    options=method_options,
    value=method_options,
    description='Methods:',
    disabled=False
)

plot_type_widget = widgets.RadioButtons(
    options=['bar', 'radar'],
    value='bar',
    description='Plot Type:',
    disabled=False
)

# Create interactive plot
interact(plot_comparison, 
        metrics=metrics_widget,
        methods=methods_widget,
        plot_type=plot_type_widget);

## Kết Luận

Từ các visualizations trên, chúng ta có thể thấy:

1. **ROUGE Scores**:
   - QLoRA đạt kết quả tốt nhất về ROUGE scores
   - LoRA và Adapter (IA3) có hiệu suất gần tương đương
   - Prompt-tuning có hiệu suất thấp hơn

2. **Resource Usage**:
   - Prompt-tuning sử dụng ít VRAM và tham số nhất
   - QLoRA tiết kiệm VRAM hơn LoRA thông thường
   - Adapter (IA3) có số lượng tham số nhỏ nhưng dùng nhiều VRAM

3. **Training Time**:
   - Prompt-tuning huấn luyện nhanh nhất
   - QLoRA và Adapter tốn thời gian huấn luyện nhiều hơn
   - LoRA có thời gian huấn luyện trung bình

### Trade-offs:
- QLoRA: Hiệu suất tốt nhất nhưng thời gian huấn luyện lâu
- LoRA: Cân bằng giữa hiệu suất và tài nguyên
- Adapter: Số tham số ít, hiệu suất tốt, nhưng dùng nhiều VRAM
- Prompt-tuning: Nhẹ và nhanh, nhưng hiệu suất thấp hơn